In [28]:
import pandas as pd

In [29]:
def cleaning_ages_step1(rawdb):

    db_data = pd.read_csv(rawdb)
    
    age_group = db_data['age_group'].tolist()
    
    clean_age_group = [i.replace('_', '-') for i in age_group]
    properly_age_group = [i.replace('juvenile', '14-25') for i in clean_age_group]

    db_data['age_group'] = properly_age_group

    return db_data

In [30]:
def cleaning_ages_step2(clean_age_df):

    age_group = clean_age_df['age_group'].tolist()

    new_age_group = []

    for i in age_group:
        if i == '14-25':
            new_age_group.append(i.replace('14-25', '18-29'))
        elif i == '26-39':
            new_age_group.append(i.replace('26-39', '30-49'))
        else:
            new_age_group.append(i.replace('40-65', '50-69'))

    clean_age_df['age_group'] = new_age_group

    return clean_age_df

In [31]:
rawdb='../data/raw/data_from_db.csv'

In [125]:
clean_age_group_column = cleaning_ages_step2(cleaning_ages_step1(rawdb))

In [122]:
def completing_country_names(countrydf):
    
    country_df = pd.read_csv(countrydf)
    
    country_names_df = country_df.rename(columns={'Unnamed: 0': 'country', '0': 'country_codes'})
    
    country_names_df['country_codes'][1] = 'GR'
    
    return country_names_df

In [123]:
countrydf = '../data/raw/country_names.csv'

In [135]:
complete_countries = completing_country_names(countrydf)

In [132]:
def merging_jobs_ages(jobsdf, ages_df):
    job_titles_df = pd.read_csv(jobsdf)
    merged_df = ages_df.merge(job_titles_df, left_on='normalized_job_code', right_on='uuid')
    cleaning_df = merged_df.drop(columns = ['normalized_job_code', 'error', 'uuid_y', 'normalized_job_title', 'parent_uuid'])
    first_almost_proper_df = cleaning_df.fillna('Unemployed')
    
    return first_almost_proper_df

In [133]:
jobsdf = '../data/raw/job_titles_df.csv'

In [136]:
jobs_and_ages = merging_jobs_ages(jobsdf, clean_age_group_column)

In [145]:
def merging_countries(countries, job_and_ages):
    
    merging_countries = countries.merge(job_and_ages, left_on='country_codes', right_on='country_code')
    cleaning_df = merging_countries.drop(columns=['country_codes', 'country_code', 'uuid_x'])
    processed_df = cleaning_df.rename(columns={'country': 'Country', 'age_group': 'Age Group', 'title': 'Job Title'})
    
    processed_df.to_csv('../data/processed/processed_df.csv')
    
    return processed_df

In [146]:
merging_countries(complete_countries, jobs_and_ages)

,Country,Age Group,Job Title
0,Belgium,50-69,Unemployed
1,Belgium,50-69,Unemployed
2,Belgium,18-29,Unemployed
3,Belgium,50-69,Unemployed
4,Belgium,30-49,Unemployed
...,...,...,...
9644,Great Britain,50-69,Data Analysis Assistant
9645,Great Britain,50-69,Data Analysis Assistant
9646,Great Britain,30-49,Data Examination Clerk
9647,Great Britain,30-49,Data Examination Clerk
